In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dcor as dc
from scipy.cluster.hierarchy import linkage, cut_tree, dendrogram
from scipy.spatial.distance import squareform
from scipy.stats import norm
from math import sqrt,  tanh, ceil, log, cos, pi, sin
import seaborn as sns
import time
from numpy.linalg import inv
from numpy import linalg as LA
import random
from IPython.display import display, Math, Latex
from scipy.io import loadmat
from scipy.linalg import pinv2, sqrtm
from scipy.optimize import minimize
from scipy.linalg import orth

In [ ]:
# import functions from necessary files 
import os
original_dir = os.getcwd()
os.chdir('..\\src\\robustOptimPack\\Utils')
from helper_funcs import *
os.chdir('..\\src\\robustOptimPack\\MVPortfolio')
from MVPortfolio_funcs import *
os.chdir('..\\src\\robustOptimPack\\HCPortfolio')
from HCPortfolio_funcs import *
os.chdir('..\\src\\robustOptimPack\\PortfolioBacktest')
from Backtest_funcs import *
os.chdir('..\\src\\robustOptimPack\\wrapping')
from wrapping_funcs import *
os.chdir(original_dir)

In [ ]:
# Import data 
crypto_prices = pd.read_csv('..\\data\\crypto_daily_close2.csv')
crypto_volumes = pd.read_csv('..\\data\\crypto_daily_volume2.csv')
tradA_prices = pd.read_csv('..\\data\\tradA.csv')
tradA_prices['Date'] = pd.to_datetime(tradA_prices['Date'])
tradA_prices.set_index('Date', inplace = True)
crypto_prices['Date'] = pd.to_datetime(crypto_prices['Date'])
crypto_volumes['Date'] = pd.to_datetime(crypto_volumes['Date'])
crypto_prices.set_index('Date', inplace = True)
crypto_volumes.set_index('Date', inplace = True)
crypto_prices = crypto_prices.loc['2015-01-01':'2021-10-31']
crypto_volumes = crypto_volumes.loc['2015-01-01':'2021-10-31']
stable_coins_names = ['USDT', 'USDC',  'BUSD', 'UST','DAI', 'UST', 'TUSD', 'USDP','USDN', 'RSR', 'FEI', 'HUSD', 'FRAX', 'LUSD', 'TRIBE', 'USDX', 'GUSD', 'XSGD', 'GUSD', 'CUSD', 'VAI', 'SUSD','EURS']
crypto_prices = crypto_prices[crypto_prices.columns.difference(stable_coins_names)]
crypto_volumes = crypto_volumes[crypto_volumes.columns.difference(stable_coins_names)]
crypto_returns = compute_returns(crypto_prices)
crypto_log_returns = compute_log_returns(crypto_prices)
tradA_returns = compute_returns(tradA_prices)
crypto_returns = crypto_returns.loc[crypto_returns.index.isin(tradA_returns.index)]
crypto_log_returns = crypto_log_returns.loc[crypto_log_returns.index.isin(tradA_returns.index)]


In [ ]:
test_window = 21
train_window = 126
n_assets = 20


In [ ]:
n_assets = 20
portfolio_equal_crypto = EqualWeightPortfolio(asset_names=list(crypto_prices.columns.values)[0:n_assets] , data=crypto_log_returns, is_price_data=False, risk_free_rate=0, period_length=1, name='EW', init_amount_invested=1)


portfolio_cov_crypto = MVPortfolio(asset_names=list(crypto_prices.columns.values)[0:n_assets], data=crypto_log_returns, is_price_data=False, risk_free_rate=0, period_length=1, name='MV',init_amount_invested=1,long_only=True)

portfolio_HRP_crypto_cor_average = HRP_portfolio(asset_names=list(crypto_prices.columns.values)[0:n_assets] , data=crypto_log_returns, is_price_data=False, risk_free_rate=0, period_length=1, name='HRP_cor', init_amount_invested=1, 
                                             cov_method = 'sample', cor_method = 'sample', linkage_method = "average")

portfolio_HRP_crypto_dcor_average = HRP_portfolio(asset_names=list(crypto_prices.columns.values)[0:n_assets] , data=crypto_log_returns, is_price_data=False, risk_free_rate=0, period_length=1, name='HRP_dcor', init_amount_invested=1, 
                                             cov_method = 'sample', cor_method = 'dcor', linkage_method = "average")

portfolio_HRP_crypto_wcor_average = HRP_portfolio(asset_names=list(crypto_prices.columns.values)[0:n_assets] , data=crypto_log_returns, is_price_data=False, risk_free_rate=0, period_length=1, name='HRP_wcor', init_amount_invested=1, 
                                             cov_method = 'sample', cor_method = 'robust_cor', linkage_method = "average")

portfolio_HRP_crypto_wdcor_average = HRP_portfolio(asset_names=list(crypto_prices.columns.values)[0:n_assets] , data=crypto_log_returns, is_price_data=False, risk_free_rate=0, period_length=1, name='HRP_wdcor', init_amount_invested=1, 
                                             cov_method = 'sample', cor_method = 'robust_dcor', linkage_method = "average")

portfolio_HERC_crypto_cor_average_sil = HERC_portfolio(asset_names=list(crypto_prices.columns.values)[0:n_assets] , data=crypto_log_returns, is_price_data=False, risk_free_rate=0, period_length=1, name='HERC_cor_sil', init_amount_invested=1, 
                                             cov_method = 'sample', cor_method = 'sample', linkage_method = "average",num_clusters_method = 'silhouette' )

portfolio_HERC_crypto_dcor_average_sil = HERC_portfolio(asset_names=list(crypto_prices.columns.values)[0:n_assets] , data=crypto_log_returns, is_price_data=False, risk_free_rate=0, period_length=1, name='HERC_dcor_sil', init_amount_invested=1, 
                                             cov_method = 'sample', cor_method = 'dcor', linkage_method = "average", num_clusters_method = 'silhouette')

portfolio_HERC_crypto_wcor_average_sil = HERC_portfolio(asset_names=list(crypto_prices.columns.values)[0:n_assets] , data=crypto_log_returns, is_price_data=False, risk_free_rate=0, period_length=1, name='HERC_wcor_sil', init_amount_invested=1, 
                                             cov_method = 'sample', cor_method = 'robust_cor', linkage_method = "average",num_clusters_method = 'silhouette')

portfolio_HERC_crypto_wdcor_average_sil = HERC_portfolio(asset_names=list(crypto_prices.columns.values)[0:n_assets] , data=crypto_log_returns, is_price_data=False, risk_free_rate=0, period_length=1, name='HERC_wdcor_sil', init_amount_invested=1, 
                                             cov_method = 'sample', cor_method = 'robust_dcor', linkage_method = "average",num_clusters_method = 'silhouette')

portfolio_cluster_sharpe_sil = max_cluster_portfolio(asset_names=list(crypto_prices.columns.values)[0:n_assets] , data=crypto_log_returns, is_price_data=False, risk_free_rate=0, period_length=1, name='cluster_sharpe', init_amount_invested=1, 
                                             cor_method = 'sample', linkage_method = "complete", num_clusters_type = 'fixed', num_clusters_method = 'fixed', max_num_clusters = 4, risk_method = "sharpe" )

portfolio_cluster_CVaR_sil = max_cluster_portfolio(asset_names=list(crypto_prices.columns.values)[0:n_assets] , data=crypto_log_returns, is_price_data=False, risk_free_rate=0, period_length=1, name='cluster_CVaR', init_amount_invested=1, 
                                             cor_method = 'sample', linkage_method = "average", num_clusters_type = 'search', num_clusters_method = 'silhouette', risk_method = "CVaR" )

portfolio_cluster_sharpe_sil_dcor = max_cluster_portfolio(asset_names=list(crypto_prices.columns.values)[0:n_assets] , data=crypto_log_returns, is_price_data=False, risk_free_rate=0, period_length=1, name='cluster_sharpe_dcor', init_amount_invested=1, 
                                             cor_method = 'dcor', linkage_method = "complete", num_clusters_type = 'fixed', num_clusters_method = 'fixed',max_num_clusters = 4, risk_method = "sharpe" )

portfolio_cluster_CVaR_sil_dcor = max_cluster_portfolio(asset_names=list(crypto_prices.columns.values)[0:n_assets] , data=crypto_log_returns, is_price_data=False, risk_free_rate=0, period_length=1, name='cluster_CVaR_dcor', init_amount_invested=1, 
                                             cor_method = 'dcor', linkage_method = "average", num_clusters_type = 'search', num_clusters_method = 'silhouette', risk_method = "CVaR" )


portfolio_cluster_sharpe_sil_wcor = max_cluster_portfolio(asset_names=list(crypto_prices.columns.values)[0:n_assets] , data=crypto_log_returns, is_price_data=False, risk_free_rate=0, period_length=1, name='cluster_sharpe_wcor', init_amount_invested=1, 
                                             cor_method = 'robust_cor', linkage_method = "complete", num_clusters_type = 'fixed', num_clusters_method = 'fixed',max_num_clusters = 4, risk_method = "sharpe" )

portfolio_cluster_CVaR_sil_wcor = max_cluster_portfolio(asset_names=list(crypto_prices.columns.values)[0:n_assets] , data=crypto_log_returns, is_price_data=False, risk_free_rate=0, period_length=1, name='cluster_CVaR_wcor', init_amount_invested=1, 
                                             cor_method = 'robust_cor', linkage_method = "average", num_clusters_type = 'search', num_clusters_method = 'silhouette', risk_method = "CVaR" )


portfolio_cluster_sharpe_sil_wdcor = max_cluster_portfolio(asset_names=list(crypto_prices.columns.values)[0:n_assets] , data=crypto_log_returns, is_price_data=False, risk_free_rate=0, period_length=1, name='cluster_sharpe_wdcor', init_amount_invested=1, 
                                             cor_method = 'robust_dcor', linkage_method = "complete", num_clusters_type = 'fixed', num_clusters_method = 'fixed',max_num_clusters = 4, risk_method = "sharpe" )

portfolio_cluster_CVaR_sil_wdcor = max_cluster_portfolio(asset_names=list(crypto_prices.columns.values)[0:n_assets] , data=crypto_log_returns, is_price_data=False, risk_free_rate=0, period_length=1, name='cluster_CVaR_wdcor', init_amount_invested=1, 
                                             cor_method = 'robust_dcor', linkage_method = "average", num_clusters_type = 'search', num_clusters_method = 'silhouette', risk_method = "CVaR" )


list_of_portfolios = [portfolio_equal_crypto, portfolio_cov_crypto, 
                    portfolio_HRP_crypto_cor_average,portfolio_HRP_crypto_dcor_average,portfolio_HRP_crypto_wcor_average,portfolio_HRP_crypto_wdcor_average,
                    portfolio_HERC_crypto_cor_average_sil,portfolio_HERC_crypto_dcor_average_sil,portfolio_HERC_crypto_wcor_average_sil,portfolio_HERC_crypto_wdcor_average_sil,
                    portfolio_cluster_sharpe_sil,
                      portfolio_cluster_CVaR_sil, 
                      portfolio_cluster_sharpe_sil_dcor, 
                      portfolio_cluster_CVaR_sil_dcor, 
                      portfolio_cluster_sharpe_sil_wcor, 
                      portfolio_cluster_CVaR_sil_wcor, 
                      portfolio_cluster_sharpe_sil_wdcor,
                      portfolio_cluster_CVaR_sil_wdcor, 
]

In [ ]:
portfolios_backtest_crypto = backtest_crypto_portfolios(list_of_portfolios,returns_data=crypto_returns,
                                                        log_returns_data = crypto_returns, tradA_data = tradA_returns,
                                                        is_price_data=False, volumes = crypto_volumes.iloc[1:,], dates = crypto_returns.index, K=n_assets, 
                                                        train_window_size = train_window, transaction_cost = 0,
                                                        test_window_size = test_window,rebalance_window_size=test_window)
portfolios_backtest_crypto.run_backtest()

In [ ]:
portfolios_backtest_crypto.out_of_sample_results().to_latex()